In [ ]:
players_df = spark.read.table("players")
display(players_df.limit(100))

In [ ]:
ronaldo_df = players_df.filter(players_df["name"].contains("Ronaldo"))
display(ronaldo_df)

In [ ]:
appearances_df = spark.read.table("appearances")

ronaldo_appearances = appearances_df.filter(appearances_df["player_id"] == "8198")
display(ronaldo_appearances.limit(5))

In [ ]:
games_df = spark.read.table("games")

display(games_df.select("game_id", "date", "home_club_name", "away_club_name", "home_club_goals", "away_club_goals").limit(50))

In [ ]:
# ronaldo_matches = ronaldo_appearances.join(
#     games_df,
#     ronaldo_appearances["game_id"] == games_df["game_id"],
#     how = "inner"
# )

# display(ronaldo_matches.select(
#     "date", "home_club_name", "away_club_name",
#     "goals", "assists", "minutes_played", "yellow_cards"
# ).orderBy("date", ascending = False ))

In [ ]:
from pyspark.sql.functions import col

# Alias both tables
a = ronaldo_appearances.alias("a")
g = games_df.alias("g")

# Join with aliases
ronaldo_matches = a.join(
    g,
    a["game_id"] == g["game_id"],
    how="inner"
)

# Clean select with unambiguous column names
display(
    ronaldo_matches.select(
        col("g.date"),
        col("g.home_club_name"),
        col("g.away_club_name"),
        col("a.goals"),
        col("a.assists"),
        col("a.minutes_played"),
        col("a.yellow_cards")
    ).orderBy(col("g.date").desc())
)


In [65]:
from pyspark.sql.functions import sum, count

ronaldo_summary = ronaldo_matches.select(
    col("a.goals"),
    col("a.assists"),
    col("a.minutes_played")
)

display(
    ronaldo_summary.agg(
        sum("goals").alias("Total Goals"),
        sum("assists").alias("Total Assists"),
        count("*").alias("Matches Played")
    )
)

StatementMeta(, 2151760b-b6ce-4f85-9e58-f0c8c9e4416b, 67, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8e83e526-d777-4517-b0db-bae89a6f862e)

In [ ]:
display(ronaldo_matches.select("g.date").count())

In [ ]:
#This analysis reflects only the subset of Ronaldo’s matches available in the Kaggle Transfermarkt dataset.
#It may not include all international, youth, or early career games.
#Dataset includes data from 2012-2022

from pyspark.sql.functions import min, max, col

display(
    ronaldo_matches.agg(
        min(col("g.date")).alias("First Match"),
        max(col("g.date")).alias("Last Match")
    )
)


In [66]:
from pyspark.sql.functions import year

ronaldo_by_year = ronaldo_matches.withColumn("year", year(col("g.date")))

goals_per_year = ronaldo_by_year.groupBy("year").agg(
    sum("a.goals").alias("total_goals")
).orderBy("year")

display(goals_per_year)

StatementMeta(, 2151760b-b6ce-4f85-9e58-f0c8c9e4416b, 68, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a7910a7c-a90c-441f-9510-f2a85a6e3716)

In [ ]:
display(
    ronaldo_by_year.filter(col("year") >= 2014).select(
        "g.date", "g.home_club_name", "g.away_club_name", "a.goals", "a.assists"
    ).orderBy(col("g.date").desc())
)


In [60]:
ronaldo_matchwise = ronaldo_matches.select(
    col("g.date").alias("Match Date"),
    col("g.home_club_name").alias("Home Team"),
    col("g.away_club_name").alias("Away Team"),
    col("a.goals").alias("Goals"),
    col("a.assists").alias("Assists"),
    col("a.minutes_played").alias("Minutes Played"),
    col("a.yellow_cards").alias("Yellow Cards"),
    col("a.red_cards").alias("Red Cards")
).orderBy(col("g.date").asc())

display(ronaldo_matchwise)

StatementMeta(, 2151760b-b6ce-4f85-9e58-f0c8c9e4416b, 62, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6654a217-4fdc-4605-b628-a15c9f401378)

In [61]:
from pyspark.sql.functions import year

ronaldo_cards = ronaldo_matches.withColumn("year", year(col("g.date")))

ronaldo_cards_summary = ronaldo_cards.groupBy("year").agg(
    sum("a.yellow_cards").alias("Yellow Cards"),
    sum("a.red_cards").alias("Red Cards")
).orderBy("year")

display(ronaldo_cards_summary)


StatementMeta(, 2151760b-b6ce-4f85-9e58-f0c8c9e4416b, 63, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c3637f7d-868b-4d7c-a53c-ab870248f21d)

In [62]:
from pyspark.sql.functions import when

ronaldo_buckets = ronaldo_matches.withColumn(
    "Play Time Range",
    when(col("a.minutes_played") < 30, "<30 mins")
    .when((col("a.minutes_played") >= 30) & (col("a.minutes_played") < 60), "30–59 mins")
    .when((col("a.minutes_played") >= 60) & (col("a.minutes_played") < 90), "60–89 mins")
    .otherwise("Full 90")
)

display(
    ronaldo_buckets.groupBy("Play Time Range").agg(count("*").alias("Match Count"))
)


StatementMeta(, 2151760b-b6ce-4f85-9e58-f0c8c9e4416b, 64, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f0a79061-5ada-4e60-8548-9593ec87b4ad)

In [64]:
ronaldo_location = ronaldo_matches.withColumn(
    "Match Type",
    when(col("g.home_club_name") == "Real Madrid Club de Fútbol", "Home")
    .otherwise("Away")
)

display(
    ronaldo_location.groupBy("Match Type").agg(
        sum("a.goals").alias("Goals"),
        sum("a.assists").alias("Assists"),
        count("*").alias("Matches")
    )
)


StatementMeta(, 2151760b-b6ce-4f85-9e58-f0c8c9e4416b, 66, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2eae9806-0f2a-4f6b-8060-18ee2d67305a)

In [1]:
df = spark.read.table("appearances_clean")
display(df)


StatementMeta(, 6d190829-12ed-4153-8061-3a093fae4153, 3, Finished, Available, Finished)

Cache file does not exists. resource=$ml, path=$/nfs4/tridenttokenlibrary/tokens/ml.token
failed to get ml token
Traceback (most recent call last):
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/synapse/ml/fabric/token_utils.py", line 156, in _get_access_token_from_token_lib
    return PyTridentTokenLibrary.get_access_token_from_cache(resource)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/trident_token_library_wrapper.py", line 71, in get_access_token_from_cache
    raise IOError(err_msg)
OSError: Cache file does not exists. resource=$ml, path=$/nfs4/tridenttokenlibrary/tokens/ml.token


SynapseWidget(Synapse.DataFrame, 866e01e4-4c20-46c7-bd39-5217051d426d)